# model file inference tracker:
**Params:**
resnet50,90/10,e12,bs8,SGD,conf:0.15,480p,labeled data,lr_sch

<!-- **Inference NB:** https://www.kaggle.com/soumya9977/learning-to-torch-fasterrcnn-infer -->

| Version | model      |  file used                      | link  | CV/LB        |
| ------- | ---------- |  ------------------------------ | ----- | ------------ |
| v9      | fasterRCNN resnet50,90/10,e12,bs8,SGD,cnf0.15,i480| fasterrcnn_resnet50_fpn-e10.pt | [NB](https://www.kaggle.com/shivamaranya/fasterrcnn-resnet50-90-10-e12-bs8-sgd-cnf0-25-i480) | 0.461/0.285  |
| v12     | fasterRCNN ................... same as above | fasterrcnn_resnet50_fpn-e9.pt  | [NB](https://www.kaggle.com/shivamaranya/fasterrcnn-resnet50-90-10-e12-bs8-sgd-cnf0-25-i480) | 0.461/~0.285 |
| v10     | fasterRCNN ................... same as above | fasterrcnn_resnet50_fpn-e11.pt | [NB](https://www.kaggle.com/shivamaranya/fasterrcnn-resnet50-90-10-e12-bs8-sgd-cnf0-25-i480) | 0.459/0.285  |
| v13     | fasterRCNN ................... same as above | fasterrcnn_resnet50_fpn-e8.pt  | [NB](https://www.kaggle.com/shivamaranya/fasterrcnn-resnet50-90-10-e12-bs8-sgd-cnf0-25-i480) | 0.460/0.288  |
| v11     | fasterRCNN ................... same as above | fasterrcnn_resnet50_fpn-e6.pt  | [NB](https://www.kaggle.com/shivamaranya/fasterrcnn-resnet50-90-10-e12-bs8-sgd-cnf0-25-i480) | 0.457/0.291  |
| v16     | fasterRCNN resnet50,90/10,e12,bs8,SGD,cnf0.15,i480,geo aug | fasterrcnn_resnet50_fpn-e6.pt  | [NB](https://www.kaggle.com/shivamaranya/fasterrcnn-train-geoaug-480p-sgd-90-10) | 0.467/0.274  |
| v17     | fasterRCNN resnet50,90/10,e20,bs8,SGD,cnf0.15,i480,color aug | fasterrcnn_resnet50_fpn-e6.pt  | [NB](https://www.kaggle.com/shivamaranya/fasterrcnn-train-coloraug-480p-sgd-90-10-e20/) | 0.407/0.382  |
| v18     | fasterRCNN resnet50,90/10,e20,bs8,SGD,cnf0.15,i480,color aug | fasterrcnn_resnet50_fpn-e20.pt  | [NB](https://www.kaggle.com/shivamaranya/fasterrcnn-train-coloraug-480p-sgd-90-10-e20/) | 0.407/0.291  |
| v20     | FasterRCNN[train]-color+geo aug-480p-SGD-90:10-e20,multi conf,new train loop,bs8 | fasterrcnn_resnet50_fpn-e11.pt  | [NB](https://www.kaggle.com/soumya9977/fasterrcnn-train-color-geo-aug-480p-sgd-90-10-e20) | 0.338/?  |
| v21     | FasterRCNN[train]-color+geo aug-480p-SGD-90:10-e20,multi conf,new train loop,bs8 | fasterrcnn_resnet50_fpn-e20.pt  | [NB](https://www.kaggle.com/soumya9977/fasterrcnn-train-color-geo-aug-480p-sgd-90-10-e20) | 0.338/0.184|
| v22     | fasterRCNN resnet50,90/10,e20,bs8,SGD,cnf0.15,i480,color aug [inf imgSize2400] | fasterrcnn_resnet50_fpn-e6.pt  | [NB](https://www.kaggle.com/shivamaranya/fasterrcnn-train-coloraug-480p-sgd-90-10-e20/) | 0.407/0.00 [problem in the code] |
| v23     | fasterRCNN resnet50,90/10,e16,bs8,AdamW,cnf0.15,i480,color aug [save_multy: future_resume]| fasterrcnn_resnet50_fpn-e7.pt  | [NB](https://www.kaggle.com/shivamaranya/fasterrcnn-train-coloraug-480p-sgd-adamw-90-10-e20) | 0.382/?  |
| v24     | fasterRCNN resnet50,90/10,e16,bs8,AdamW,**cnf0.1**,i480,color aug [save_multy: future_resume]| fasterrcnn_resnet50_fpn-e7.pt  | [NB](https://www.kaggle.com/shivamaranya/fasterrcnn-train-coloraug-480p-sgd-adamw-90-10-e20) | 0.382/?  |

In [1]:
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from albumentations.pytorch.transforms import ToTensorV2
import albumentations as A
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision import transforms
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

WEIGHTS_FILE = "../input/fasterrcnntrainv1coloraug480padamw9010e20/FasterRCNN_resnet50/fasterrcnn_resnet50_fpn-e7.pt"

In [2]:
def get_model(mode="normal"):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)
    num_classes = 2  # 1 class (starfish) + background

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # Load the trained weights
    if mode == "multi":
        checkpoint = torch.load(WEIGHTS_FILE)
        # load model weights state_dict
        model.load_state_dict(checkpoint['model_state_dict'])
    else:
        model.load_state_dict(torch.load(WEIGHTS_FILE))
    
    model.eval()

    model = model.to(device)
    return model

model = get_model(mode="multi")

In [3]:
detection_threshold = 0.1

# transform = A.Compose([
#         A.Resize(2400, 2400, interpolation=1, p=1.0),
#         ToTensorV2(p=1.0)
#     ])

In [4]:
def format_prediction_string(boxes, scores):
    # Format as specified in the evaluation page
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.2f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)


def predict(model, pixel_array):
    # Predictions for a single image
    
    # Apply all the transformations that are required
    pixel_array = pixel_array.astype(np.float32) / 255.
#     print()
    tensor_img = ToTensorV2(p=1.0)(image=pixel_array)['image'].unsqueeze(0)
#     print(ToTensorV2(p=1.0)(image=pixel_array)['image'].shape)
    # Get predictions
    with torch.no_grad():
        outputs = model(tensor_img.to(device))[0]
    
    # Move predictions to cpu and numpy
    boxes = outputs['boxes'].data.cpu().numpy()
    scores = outputs['scores'].data.cpu().numpy()
    
    # Filter predictions with low score
    boxes = boxes[scores >= detection_threshold].astype(np.int32)
    scores = scores[scores >= detection_threshold]
    
    # Go back from x_min, y_min, x_max, y_max to x_min, y_min, w, h
    boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
    boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
  
    # Format results as requested in the Evaluation tab
    return format_prediction_string(boxes, scores)

In [5]:
import greatbarrierreef
env = greatbarrierreef.make_env()
iter_test = env.iter_test() 

for (pixel_array, df_pred) in iter_test:  # iterate through all test set images
    df_pred['annotations'] = predict(model, pixel_array)
    env.predict(df_pred)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


In [6]:
# test_img = np.load("../input/tensorflow-great-barrier-reef/example_test.npy")
# # test_img = torch.from_numpy(test_img)
# # test_img = test_img.permute(0,3,1,2) 
# # test_img_1 = test_img[0,:,:,:]
# test_img.shape

In [7]:
# pixel_array = test_img[0,:,:,:].astype(np.float32)
# tensor_img = ToTensorV2(p=1.0)(image=pixel_array)['image'].unsqueeze(0)
# # tensor_img.shape
# # Get predictions
# with torch.no_grad():
#     outputs = model(tensor_img.to(device))[0]
# print(outputs)